In [26]:
import symforce.symbolic as sf
from symforce import symbolic as sm
from symforce.geo import Pose3
import math

print("Step 1: Importing libraries and defining joint axes")
# Define joint axes
joint_axes = [
    sf.V3(0, 0, 1),
    sf.V3(0, 1, 0),
    sf.V3(0, 1, 0),
    sf.V3(0, 1, 0),
    sf.V3(0, 0, -1),
    sf.V3(0, 1, 0)
]
print(f"Joint axes: {joint_axes}")

print("\nStep 2: Defining constants")
# Define constants
W1, W2, L1, L2, H1, H2 = sf.symbols('W1 W2 L1 L2 H1 H2')
print(f"Constants: {W1}, {W2}, {L1}, {L2}, {H1}, {H2}")

print("\nStep 3: Defining M matrix")
# Define M matrix
M = sf.Matrix44([
    [1, 0, 0, L1 + L2],
    [0, 1, 0, W1 + W2],
    [0, 0, 1, H1 - H2],
    [0, 0, 0, 1]
])
print(f"M matrix:\n{M}")

print("\nStep 4: Defining points on the joint axes")
# Define points on the joint axes
joint_points = [
    sf.V3(0, 0, 0),
    sf.V3(0, 0, H1),
    sf.V3(L1, 0, H1),
    sf.V3(L1 + L2, 0, H1),
    sf.V3(L1 + L2, W1, 0),
    sf.V3(L1 + L2, 0, H1 - H2)
]
print(f"Joint points: {joint_points}")

print("\nStep 5: Defining symbolic joint angles")
# Define symbolic joint angles
theta = sf.symbols('theta1:7')
print(f"Joint angles: {theta}")

print("\nStep 6: Computing screw axes")
# Compute screw axes
S = []
for axis, point in zip(joint_axes, joint_points):
    v = -axis.cross(point)
    S.append(sf.V6(axis.x, axis.y, axis.z, v.x, v.y, v.z))
print(f"Screw axes: {S}")

print("\nStep 7: Converting M to a Pose3 object")
# Convert M to a Pose3 object
M_pose = Pose3(sf.Rot3.from_rotation_matrix(M[:3, :3]), sf.V3(M[:3, 3]))
print(f"M as Pose3: {M_pose}")

print("\nStep 8: Computing forward kinematics")
# Compute forward kinematics
T = M_pose
for i in range(6):
    T = Pose3.from_tangent(S[i] * theta[i]) * T
print("Symbolic forward kinematics:")
print(T)

print("\nStep 9: Substituting values")
# Substitute values
subs_dict = {
    W1: 0.109, W2: 0.082, L1: 0.425, L2: 0.392, H1: 0.089, H2: 0.095,
    theta[0]: 0, theta[1]: -math.pi/2, theta[2]: 0, theta[3]: 0, theta[4]: math.pi/2, theta[5]: 0
}
T_numeric = T.subs(subs_dict)
print("Numeric result:")
print(T_numeric)

print("\nStep 10: Defining expected result")
# Expected result
expected_T = sf.Matrix44([
    [0, -1, 0, 0.095],
    [1, 0, 0, 0.109],
    [0, 0, 1, 0.988],
    [0, 0, 0, 1]
])
expected_T = Pose3(sf.Rot3.from_rotation_matrix(expected_T[:3, :3]), sf.V3(expected_T[:3, 3]))
print("Expected result:")
print(expected_T)

print("\nStep 11: Comparing results")
# Compare results
diff = sf.V6(T_numeric.to_tangent()) - sf.V6(expected_T.to_tangent())
print(f"Difference norm: {(diff).norm()}")

Step 1: Importing libraries and defining joint axes
Joint axes: [[0]
[0]
[1]
, [0]
[1]
[0]
, [0]
[1]
[0]
, [0]
[1]
[0]
, [0]
[0]
[-1]
, [0]
[1]
[0]
]

Step 2: Defining constants
Constants: W1, W2, L1, L2, H1, H2

Step 3: Defining M matrix
M matrix:
[1, 0, 0, L1 + L2]
[0, 1, 0, W1 + W2]
[0, 0, 1, H1 - H2]
[0, 0, 0, 1]


Step 4: Defining points on the joint axes
Joint points: [[0]
[0]
[0]
, [0]
[0]
[H1]
, [L1]
[0]
[H1]
, [L1 + L2]
[0]
[H1]
, [L1 + L2]
[W1]
[0]
, [L1 + L2]
[0]
[H1 - H2]
]

Step 5: Defining symbolic joint angles
Joint angles: (theta1, theta2, theta3, theta4, theta5, theta6)

Step 6: Computing screw axes
Screw axes: [[0]
[0]
[1]
[0]
[0]
[0]
, [0]
[1]
[0]
[-H1]
[0]
[0]
, [0]
[1]
[0]
[-H1]
[0]
[L1]
, [0]
[1]
[0]
[-H1]
[0]
[L1 + L2]
, [0]
[0]
[-1]
[-W1]
[L1 + L2]
[0]
, [0]
[1]
[0]
[-(H1 - H2)]
[0]
[L1 + L2]
]

Step 7: Converting M to a Pose3 object
M as Pose3: <Pose3 R=<Rot3 <Q xyzw=[0, 0, 0, 1]>>, t=(L1 + L2, W1 + W2, H1 - H2)>

Step 8: Computing forward kinematics
Symbolic f

In [27]:
import sympy as sp

print("\nStep 11: Comparing results")
# Compare results
diff = sf.V6(T_numeric.to_tangent()) - sf.V6(expected_T.to_tangent())
diff_norm_symbolic = diff.norm()
print(f"Symbolic difference norm: {diff_norm_symbolic}")

# Convert the SymForce expression to a SymPy expression
diff_norm_sympy = sp.sympify(str(diff_norm_symbolic))

# Evaluate the SymPy expression numerically
diff_norm_numeric = diff_norm_sympy.evalf()

print(f"Numeric difference norm: {diff_norm_numeric}")


Step 11: Comparing results
Symbolic difference norm: sqrt(4.01711976466577 + (-1.20919957615615 - sqrt(2)*(1 + 2*min(0, sign(sqrt(2))))*acos(min(abs((1/2)*sqrt(2)), 1.0))/sqrt(1 - min(abs((1/2)*sqrt(2)), 1.0)**2))**2)


TypeError: Value after * must be an iterable, not Pow

In [ ]:

# # Define joint angles as symbols
# theta1 = sf.Symbol("theta1")
# theta2 = sf.Symbol("theta2")

# # Define link lengths as symbols (or constants if known)
# L1 = sf.Symbol("L1")
# L2 = sf.Symbol("L2")

# # Define joint twists
# twist1 = sf.V6(0, 0, 1, 0, 0, 0)  # Rotation around z-axis
# twist2 = sf.V6(0, 0, 1, L1, 0, 0) # Rotation around z-axis, shifted by L1

# print(f"Twist1: {twist1}")
# print(f"Twist2: {twist2}")

# # Build forward kinematics
# T01 = sf.Pose3(sf.Rot3.from_angle_axis(theta1, sf.V3(0, 0, 1)), sf.V3(0, 0, 0))
# T12 = sf.Pose3(sf.Rot3.from_angle_axis(theta2, sf.V3(0, 0, 1)), sf.V3(L1, 0, 0))
# T02 = T01.compose(T12)


# print(f"T01: {T01}")
# print(f"T12: {T12}")
# print(f"T02: {T02}")

In [14]:
# from symforce import symbolic as sm
# from symforce.geo import Pose3

# def skew_symmetric(v):
#     """
#     Compute the skew-symmetric matrix of a 3D vector.
    
#     Args:
#         v (sf.V3): 3D vector
    
#     Returns:
#         sf.Matrix33: Skew-symmetric matrix
#     """
#     return sf.Matrix33([
#         [0, -v[2], v[1]],
#         [v[2], 0, -v[0]],
#         [-v[1], v[0], 0]
#     ])

# def se3_exp(xi, theta):
#     """
#     Computes the exponential map of an element in se(3) to SE(3).

#     Args:
#       xi: 6-dimensional vector representing the twist coordinates.
#       theta: Scalar representing the joint variable.

#     Returns:
#       Pose3 object representing the corresponding element in SE(3).
#     """
#     omega = sf.V3(xi[:3])
#     v = sf.V3(xi[3:])

#     omega_hat = skew_symmetric(omega)  # Get the skew-symmetric matrix of omega
#     omega_norm = omega.norm()

#     if omega_norm == 0:  # Handle pure translation case
#         R = sf.Rot3.identity()
#         t = v * theta
#     else:
#         R = sf.Rot3.from_tangent(omega * theta)
#         # Use matrix multiplication for squaring omega_hat
#         t = (sf.Matrix33.eye(3) * theta + (1 - sf.cos(omega_norm * theta)) / omega_norm**2 * omega_hat +
#              (omega_norm * theta - sf.sin(omega_norm * theta)) / omega_norm**3 * omega_hat * omega_hat) * v

#     return Pose3(R, t)

# # Example usage (unchanged)
# theta1, theta2 = sm.symbols('theta1 theta2')

# xi1 = sf.V6(0, 0, 1, 0, 0, 0)  # Twist for rotation around z-axis
# xi2 = sf.V6(0, 1, 0, 0, 0, 1)  # Twist for rotation around y-axis with translation along x-axis

# g1 = se3_exp(xi1, theta1)
# g2 = se3_exp(xi2, theta2)

# g = g1 * g2  # PoE calculation


# # Get the tangent representation
# tangent = g.to_tangent()

# # Check the type of the tangent
# print("Type of tangent:", type(tangent))

# # If it's a list or sf.Vector, convert it to a symbolic vector
# if isinstance(tangent, (list, sf.Vector6)):
#     tangent_vec = sf.V6(tangent)
#     print("Tangent vector:")
#     print(tangent_vec)
    
#     # Try to simplify each element separately
#     simplified_tangent2 = sf.V6([sf.simplify(elem) for elem in tangent_vec])
#     print("Simplified tangent vector:")
#     print(simplified_tangent2)
# else:
#     # If it's already a symbolic expression, simplify directly
#     simplified_tangent2 = sf.simplify(tangent)
#     print("Simplified tangent:")
#     print(simplified_tangent2)

# #print size of the tangent
# len(simplified_tangent2)


symbolic.simplify():563 WARNING -- Converting to sympy to use .simplify


Type of tangent: <class 'list'>
Tangent vector:
[-2*theta2*theta1*(1 + 2*min(0, sign(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2)))))*acos(min(abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))), 1.0))*sin((1/2)*sqrt(theta1**2))*sin((1/2)*sqrt(theta2**2))/(sqrt(1 - min(abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))), 1.0)**2)*sqrt(theta1**2)*sqrt(theta2**2))]
[2*theta2*(1 + 2*min(0, sign(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2)))))*acos(min(abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))), 1.0))*cos((1/2)*sqrt(theta1**2))*sin((1/2)*sqrt(theta2**2))/(sqrt(1 - min(abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))), 1.0)**2)*sqrt(theta2**2))]
[2*theta1*(1 + 2*min(0, sign(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2)))))*acos(min(abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))), 1.0))*sin((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))/(sqrt(1 - min(abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))), 1.0)**2

symbolic.simplify():563 WARNING -- Converting to sympy to use .simplify
symbolic.simplify():563 WARNING -- Converting to sympy to use .simplify
symbolic.simplify():563 WARNING -- Converting to sympy to use .simplify
symbolic.simplify():563 WARNING -- Converting to sympy to use .simplify
symbolic.simplify():563 WARNING -- Converting to sympy to use .simplify


Simplified tangent vector:
[Piecewise((-4*theta2*theta1*acos((1/2)*(1.0 + abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))) - abs(1.0 - abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))))))*sin((1/2)*sqrt(theta1**2))*sin((1/2)*sqrt(theta2**2))/(sqrt(theta1**2)*sqrt(theta2**2)*sqrt(4 - (1 + abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))) - abs(1.0 - abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2)))))**2)), 0 == cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))), (4*theta2*theta1*(-cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2)) + abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2)))*(-1 + abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))/abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))))))*acos((1/2)*(1.0 + abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))) - abs(1.0 - abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))))))*sin((1/2)*sqrt(theta1**2))*sin((1/2)*sqrt(theta2**2))/(abs(cos((1/2)*sqrt(theta1**

6

In [18]:

# # Example usage
# theta1, theta2 = sm.symbols('theta1 theta2')

# xi1 = sf.V6(0, 0, 1, 0, 0, 0)  # Twist for rotation around z-axis
# xi2 = sf.V6(0, 1, 0, 0, 0, 1)  # Twist for rotation around y-axis with translation along x-axis

# g1 = Pose3.from_tangent(xi1 * theta1)
# g2 = Pose3.from_tangent(xi2 * theta2)

# g = g1 * g2  # PoE calculation

# tangent_vector = g.to_tangent()  # Get the twist vector as a list
# simplified_tangent = sf.V6([sf.simplify(v) for v in tangent_vector])  # Simplify each element

# print(simplified_tangent)  # Print the simplified twist vector

symbolic.simplify():563 WARNING -- Converting to sympy to use .simplify
symbolic.simplify():563 WARNING -- Converting to sympy to use .simplify
symbolic.simplify():563 WARNING -- Converting to sympy to use .simplify
symbolic.simplify():563 WARNING -- Converting to sympy to use .simplify
symbolic.simplify():563 WARNING -- Converting to sympy to use .simplify
symbolic.simplify():563 WARNING -- Converting to sympy to use .simplify


[Piecewise((-4*theta2*theta1*acos((1/2)*(1.0 + abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))) - abs(1.0 - abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))))))*sin((1/2)*sqrt(theta1**2))*sin((1/2)*sqrt(theta2**2))/(sqrt(theta1**2)*sqrt(theta2**2)*sqrt(4 - (1 + abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))) - abs(1.0 - abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2)))))**2)), 0 == cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))), (4*theta2*theta1*(-cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2)) + abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2)))*(-1 + abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))/abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))))))*acos((1/2)*(1.0 + abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))) - abs(1.0 - abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**2))))))*sin((1/2)*sqrt(theta1**2))*sin((1/2)*sqrt(theta2**2))/(abs(cos((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta2**

[0]
[0]
[0]
[1.0*(-1 + cos(1.0*theta2))*cos(sqrt(theta1**2))]
[-2.0*theta1*(1 - cos(1.0*theta2))*sin((1/2)*sqrt(theta1**2))*cos((1/2)*sqrt(theta1**2))/sqrt(theta1**2)]
[theta2 - 1.0*sin(1.0*theta2)]

In [ ]:
# Define symbolic joint angles
theta = sf.symbols('theta1:7')

NotImplementedError: Unsupported type: "<class 'symforce.geo.matrix.Matrix44'>"